In [1]:
# 文本分类实例
# 1 导入相关包
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/Users/wuhaoliu/micromamba/envs/transformers/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2 加载数据
import pandas as pd
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
## 数据处理，去除空值
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
# 3 创建 Dataset
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self)-> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self,index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [5]:
dataset =MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


In [6]:
# 4 划分数据集
from torch.utils.data import random_split

trainset, validset = random_split(dataset,lengths=[0.9,0.1])
len(trainset),len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('设施非常旧，周边环境也不好，最奇怪是房间内没有暖气，只有挂壁空调，要知道，这种空调室外-5度时候，不会工作的，洛阳可是在淮河以北了。', np.int64(0))
('该楼是目前乌鲁木齐标志建筑.交通位置非常突出.出行非常方便.周边环境非常好.红山.人民公园近在咫尺.二道桥仅四站路.非常好.有缘下次仍入住............', np.int64(1))
('很不错的司星酒店，已经是为我们的贵宾客户第四次预订了，预定的行政豪华房，非常不错，只是房间略显小了些，但总体很好，服务业很好！值得推荐！', np.int64(1))
('房间象80年代的路边小旅馆.通往3楼餐厅的路上,污水横流,黑色的污渍随处可见.房间连电吹风都没有!也没有免费的矿泉水和茶叶包!电脑桌下面的地毯上黑色的污渍到处都是!!更好笑的是,有时打51(前台)后得到的回复是你找52(客房部);然后52又说你找51!!整体而言,还是国企的做法.', np.int64(0))
('这家酒店在滨州算是最好的了。整体服务水准还不错。房间一般，没有留下太深的印象，自助餐还可以，值得一吃。', np.int64(1))
('装修前卫，但是绝对不够精致，甚至有低俗在里面，不耐看！包括大堂和房间，咋不注意，还以为进了装修档次不够的KTV。房间感觉脏兮兮的！性价比低！', np.int64(0))
('如果你是用携程订的，他们会告诉你不要给携程讲住过酒店。这个酒店就是便宜，便宜无好货也是有道理的。下次不会去住了宾馆反馈2008年5月12日：您好，我是如家七斗星沙坪坝店店助，非常抱歉给您留下如此不好的印象，我为您受到的如此遭遇给您真诚的道歉。因为我店前期正在进行人员调整，前台服务员都是新招的员工，由于我们对他们的培训不够，以至于服务员在工作中的服务很不到位，确实犯了很多错误，经过这一个月的严格培训，现在酒店的服务水平已有大幅提升，各种工作已有很大进步，非常欢迎您再次入住我们酒店，相信这次肯定会留下美好的印象，真诚的邀请您。最后再谢谢您给我们提出意见，没有您这样的客人，我们的服务将不能提高，非常感谢。', np.int64(0))
('出于对海航系统的好感，虽条件并不出众，早餐简陋，（同等价格可以选择宝华大酒店、康年皇冠大酒店等很多条件更好的酒店）但仍第二次入住，提前预订同样房型（豪华商务大床房），价格也一样；但入住时

In [13]:
# 5 创建 Dataloder
import torch
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("rbt3")
def collate_func(batch):
    text,labels = [],[]
    for item in batch:
        text.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(text, max_length=128,padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"]=torch.tensor(labels)
    return inputs  
trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
validloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)

In [14]:
next(enumerate(trainloader))[1]

{'input_ids': tensor([[ 101, 3025, 4923,  ...,    0,    0,    0],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 4895, 4125,  ...,    0,    0,    0],
        ...,
        [ 101,  855, 5390,  ...,    0,    0,    0],
        [ 101, 6821, 2157,  ..., 5168, 8024,  102],
        [ 101, 2523, 1962,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 1, 1, 1, 0, 0, 1])}

In [23]:
# 6 创建模型和优化器
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3")
if torch.mps.is_available():
    model.to('cpu')
optimizer = Adam(model.parameters(),lr=2e-5) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# 7 训练与验证
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.mps.is_available():
                batch = {k:v.to("cpu") for k,v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validloader.dataset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.mps.is_available():
                batch = {k:v.to("cpu") for k,v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"epoch: {ep}, step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep:{ep},acc:{acc}")

In [ ]:
train()

/Users/wuhaoliu/micromamba/envs/transformers/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


epoch: 0, step: 0, loss: 0.7273265719413757
epoch: 0, step: 100, loss: 0.4997824728488922
epoch: 0, step: 200, loss: 0.38392189145088196
ep:0,acc:0.873711347579956
epoch: 1, step: 300, loss: 0.2474943846464157
epoch: 1, step: 400, loss: 0.35979530215263367
ep:1,acc:0.8775773048400879
epoch: 2, step: 500, loss: 0.11195611208677292


In [21]:
# 8 模型预测
sen = "我觉得这家酒店不错，饭有点好吃！"
ids2label = {0:"差评！", 1:"好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k:v.to(model.device) for k,v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"predict: {pred}")
    print(f"输入: {sen}\n模型预测结果为： {ids2label.get(pred.item())}")

predict: tensor([1], device='mps:0')
输入: 我觉得这家酒店不错，饭有点好吃！
模型预测结果为： 好评！


In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer,device=0)

In [ ]:
pipe(sen)